# Pandas I

### 0. Připravte si data

1. Stáhněte si soubor https://datahub.io/sports-data/english-premier-league/r/season-1819.csv
1. Umístěte jej do složky data

### 1. Načtěte data set `season-1819_csv.csv` a provedte následující úpravy:

1. načtěte pouze prvních 17 sloupců
1. zahoďte sloupce Div, FTR a HTR
1. sloupec Date převeďte na datetime
1. přejmenujte sloupce:
  * FTHG: FullTimeHomeGoals
  * FTAG: FullTimeAwayGoals
  * HTHG: HalfTimeHomeGoals
  * HTAG: HalfTimeAwayGoals
  * HS: HomeShots
  * AS: AwayShots
  * HST: HomeShotsTarget
  * AST: AwayShotsTarget
  * HF: HomeFauls
  * AF: AwayFauls

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
DATA_FOLDER = 'data'
DATA_SET = 'season-1819_csv.csv'



In [ ]:
import os

matches = (
    pd.read_csv(os.path.join(DATA_FOLDER, DATA_SET), parse_dates=['Date'], usecols=range(0,17))
    .drop(['Div', 'FTR', 'HTR'], axis='columns')
    .rename(columns={
        'FTHG': 'FullTimeHomeGoals',
        'FTAG': 'FullTimeAwayGoals',
        'HTHG': 'HalfTimeHomeGoals',
        'HTAG': 'HalfTimeAwayGoals',
        'HS': 'HomeShots',
        'AS': 'AwayShots',
        'HST': 'HomeShotsTarget',
        'AST': 'AwayShotsTarget',
        'HF': 'HomeFauls',
        'AF': 'AwayFauls'
    })
)

matches.head()

### 2. Zjistěte jaké datové typy mají všechny sloupečky, zda data set obsahuje nějaké NaN hodnoty a jaké jsou základní statistiky číselných sloupců.

In [ ]:
matches.info()

In [ ]:
matches.describe().transpose()

### 3. Zjistěte kolik různých týmů hraje anglickou ligu? Jaké týmy to jsou?

In [ ]:
teams = matches.HomeTeam.unique().tolist()
teams

In [ ]:
len(teams)

In [ ]:
matches.head()

### 4. Přidejte sloupečky `HomeShotAccuracy` a `AwayShotAccuracy`, které říkají jaké procento střeleckých pokusů (Shots) nakonec mířilo na bránu (ShotsTarget). Vizualizujte rozdělení hodnot obou sloupců.

In [ ]:
plt.rcParams['figure.figsize'] = (24, 6)  # redefine the default size of the plots
plt.style.use('ggplot')  # redefine default color scheme



In [ ]:
def shot_accuracy(shots: pd.Series, shots_on_target: pd.Series) -> pd.Series:
    return shots_on_target / shots * 100

matches['HomeShotAccuracy'] = shot_accuracy(matches.HomeShots, matches.HomeShotsTarget)
matches['AwayShotAccuracy'] = shot_accuracy(matches.AwayShots, matches.AwayShotsTarget)

matches[['HomeShots', 'HomeShotsTarget', 'HomeShotAccuracy', 'AwayShotAccuracy']].head()

In [ ]:
matches['HomeShotAccuracy'].plot.hist(title='Home team shot accuracy distribution');
plt.show()
matches['AwayShotAccuracy'].plot.hist(title='Away team shot accuracy distribution', color='steelblue');

In [ ]:
matches[['HomeShotAccuracy', 'AwayShotAccuracy']].plot.box(title='Shot accuracy distribution', figsize=(24, 10));
plt.ylabel('Accuracy [%]');

### 5. Vytvořte sloupeček `FullTimeWinner`, který obsahuje:

- `H`, pokud zvítězil `HomeTeam`
- `A`, pokud zvítězil `AwayTeam`
- `D`, pokud zápas skončil remízou

**Poznámka: Zatím neznáme apply, ale známe loc. Použijte tedy šikovně loc.** 

In [ ]:
matches['FullTimeWinner'] = 'D'

is_home_winner = matches.FullTimeHomeGoals > matches.FullTimeAwayGoals
is_away_winner = matches.FullTimeHomeGoals < matches.FullTimeAwayGoals

matches.loc[is_home_winner, 'FullTimeWinner'] = 'H'
matches.loc[is_away_winner, 'FullTimeWinner'] = 'A'

matches[['FullTimeHomeGoals', 'FullTimeAwayGoals', 'FullTimeWinner']].head(10)

### 6. Řekněme, že nás zajímá tým West Ham. Zjistětě, kolik průměrně vsítí gólů v domácích zápasech, které vyhraje a kolik v domácích zápasech, které prohraje.

In [ ]:
selected_team = 'West Ham'

is_team_home_match = matches.HomeTeam == selected_team
is_home_team_win = matches.FullTimeWinner == 'H'

win_goals_mean = matches.loc[is_team_home_match & is_home_team_win, 'FullTimeHomeGoals'].mean()
lost_goals_mean = matches.loc[is_team_home_match & ~is_home_team_win, 'FullTimeHomeGoals'].mean()

f"Mean of goals scored during home victory: {win_goals_mean:.2f}, defeat: {lost_goals_mean:.2f}"

### 7. Jaký je median faulů, střel na branku a vstřelených gólů West Ham pro předchozích pět domácích zápasů? Jak se tyto statistiky vyvíjí v čase? Vizualizujte je.

In [ ]:
(
    matches
    .loc[is_team_home_match, ['Date', 'HomeFauls', 'HomeShotsTarget', 'FullTimeHomeGoals']]
    .set_index('Date')
    .rolling(5).median()
    .dropna()
    .plot(title='Trend of last 5 matches')
);

### 8. Je domácí prostředí výhodou?

**Poznámka: Součástí řešení je zamyslet se, jakou formou tuto otázku zodpovědět a jaké předpoklady si mohu dovolit udělat.** 

In [ ]:
(
    matches
    .FullTimeWinner
    .value_counts()
    .div(len(matches))  # normalize counts
    .plot.bar(title='Result class counts')
)
plt.ylabel('Match count');

### 9. Kolik který rozhodčí odpískal za sezónu průměrně faulů za zápas? Seřaďte je od těch nejpřísnějších.

In [ ]:
(
    matches
    .assign(Fauls=matches.HomeFauls + matches.AwayFauls)
    .groupby('Referee')
    .mean()
    ['Fauls']
    .sort_values(ascending=False)
)

### 10. (BONUS) Budu víc vyhrávat, když budu víc faulovat?

In [ ]:
# Discuss the solution and implement it with the students